In [1]:
import os 
import glob 

import numpy as np
import pandas as pd
import xarray as xr
import netCDF4
from math import sqrt

from metpy.calc import wind_speed
from metpy.calc import wind_direction 
from metpy.units import units

from scipy import interpolate

import matplotlib.pyplot as plt
from   matplotlib.lines import Line2D
import seaborn as sns

sns.set_theme(style="whitegrid")

# Functions

## Read in Data 

In [2]:
def prep_wrf_data(fpath, var_name_dict, site):
    df = pd.read_pickle(fpath)
    df['T2'] = df['T2'] - 273  # Convert C to K
    df.index = df.index - pd.Timedelta(hours = 8)  # UTC to PST
    df = get_wind(df)  # Calc wind direction/speed
    df.rename(columns=var_name_dict, inplace=True)
    
    df.attrs['site'] = site
    df.attrs['source'] = 'wrf'
    
    return df

def prep_obs_data(fpath, site):
    df = pd.read_csv(fpath, index_col='time (PST)')
    df.index = pd.to_datetime(df.index)
    
    df.attrs['site'] = site
    df.attrs['source'] = 'obs'
    
    return df

## Calculations

In [3]:
def get_wind(df): 
    u = df['U10'].values * units('m/s')
    v = df['V10'].values * units('m/s')

    df['WND_SPD'] = wind_speed(u,v)
    df['WND_DIR'] = wind_direction(u, v, convention='from')
    
    return df

def quantile_map(wrf_series, obs_series):
    quantiles = np.arange(.1, 1, .1)

    obs_quantiles = obs_series.quantile(quantiles)
    wrf_quantiles = wrf_series.quantile(quantiles)
    
    correction_function = interpolate.interp1d(wrf_quantiles, obs_quantiles,
                                           bounds_error=False, fill_value="extrapolate")
    corrected = correction_function(wrf_series)
    
    return corrected

# Read in and Prepare Data

In [4]:
var_name_dict = {
    'DFGDP' : 'fog drip',
    'T2' : 'air temperature (C)',
    'RH' : 'relative humidity (%)',
    'WND_SPD' : 'wind speed (m/s)',
    'WND_DIR' : 'wind direction (deg)'
}

#### WRF file paths ####
wdatadir = '/home/patmccornack/data/wrf_extracted/single_pixel'
wsauc_fname = 'wrf_sauc_2003-2010.pkl'
wupem_fname = 'wrf_upem_2003-2010.pkl'
wnrs_sci_fname = 'wrf_nrs_sci_2014-2019.pkl'

#### Obs file paths ####
odatadir = '/home/patmccornack/data/preprocessed'
osauc_fname = 'SAUC_hourly.csv'
oupem_fname = 'UPEM_hourly.csv'
onrs_sci_fname = 'NRS_SCI_hourly.csv'

#### Read in data ####
wsauc = prep_wrf_data(os.path.join(wdatadir, wsauc_fname), var_name_dict, site='Sauces Canyon')
wupem = prep_wrf_data(os.path.join(wdatadir, wupem_fname), var_name_dict, site='Upper Embudo Canyon')
wnrs_sci = prep_wrf_data(os.path.join(wdatadir, wnrs_sci_fname), var_name_dict, site='NRS SCI')

osauc = prep_obs_data(os.path.join(odatadir, osauc_fname), site='Sauces Canyon')
oupem = prep_obs_data(os.path.join(odatadir, oupem_fname), site='Upper Embudo Canyon')
onrs_sci = prep_obs_data(os.path.join(odatadir, onrs_sci_fname), site='NRS SCI')

In [12]:
# Join all into a single dataframe
df_list = [wsauc, wupem, wnrs_sci, osauc, oupem, onrs_sci]
for df in df_list:
    df['site'] = df.attrs['site']
    df['source'] = df.attrs['source']

df = pd.concat(df_list, join='inner', axis=0)
df.tail(2)

,air temperature (C),relative humidity (%),wind speed (m/s),wind direction (deg),site,source
2019-12-31 22:00:00,13.213333,51.555000,2.159500,19.641667,NRS SCI,obs
2019-12-31 23:00:00,13.298333,52.808333,2.273167,78.286667,NRS SCI,obs


# Define output file paths

In [13]:
outdir = "~/data/wrf_extracted/quantile_mapped"
pd.to_pickle(df.loc[df['source']=='wrf'], os.path.join(outdir, "raw_wrf_data.pkl"))

# Perform quantile mapping

In [7]:
var_list = ['air temperature (C)', 'relative humidity (%)', 'wind speed (m/s)', 'wind direction (deg)']

In [8]:
# Map
qmwsauc = wsauc[var_list].copy()
for var in var_list:
    qmwsauc[var] = quantile_map(qmwsauc[var], osauc[var])

qmwupem = wupem[var_list].copy()
for var in var_list:
    qmwupem[var] = quantile_map(qmwupem[var], oupem[var])
    
qmwnrs_sci = wnrs_sci[var_list].copy()
for var in var_list:
    qmwnrs_sci[var] = quantile_map(qmwnrs_sci[var], onrs_sci[var])
    
    
# Save out
qmwsauc.to_pickle(os.path.join(outdir, wsauc_fname))
qmwupem.to_pickle(os.path.join(outdir, wupem_fname))
qmwnrs_sci.to_pickle(os.path.join(outdir, wnrs_sci_fname))